# **Gravitational Wave Analysis** Part 2 
### SHUBHAM 2K19/EP/093
In this notebook, I will analyse the timeseries data of the G-waves and compare the frequency spectrum that are observed at different LIGO observatories.

---

## Installing the required modules
* gwpy
* plotly

Execute these commands only when using Google Colab.

In [1]:
! pip install -q 'gwpy==1.0.1'
! pip install -q plotly

     |████████████████████████████████| 1.4 MB 29.8 MB/s 
     |████████████████████████████████| 51 kB 5.1 MB/s 
     |████████████████████████████████| 55 kB 2.6 MB/s 
     |████████████████████████████████| 3.6 MB 57.3 MB/s 


---
## Importing the modules
---
`numpy` as `np` for array manipulation.\
`scipy` as `sc` for signal processing.\
`plotly.graph_objs` as `go` for plotting.\
`gwosc.datasets` as `gd` for quering datasets.\
`gwpy.timeseries` as `gpt` for quering timeseries data.



In [2]:
import gwpy
import numpy as np
import scipy as sc
from plotly import graph_objs as go
from gwosc import datasets as gd
from gwosc import catalog as ct
from gwpy import timeseries as gpt, time as gtm

First, let's plot a simple curve using plotly.

In [3]:
x = np.linspace(0, 2*np.pi, 100)
fig = go.Figure(
    data = go.Scatter(
        x = x,
        y = np.sin(x),
        mode = 'lines',
        line = dict(
            shape = 'spline',
            color = 'darkcyan'
        )
    ),
    layout = go.Layout(
        title = 'Simple Plot',
        xaxis = dict(
            title = 'X'
        ),
        yaxis = dict(
            title = 'Y'
        )
    )
)
fig.show()

## Handling data in the time domain
---
### Finding open data
In part 1, it is already demonstrated that `gwosc` module can be used to query for what data are available on GWOSC. Now, I'll try to read some open data, let's try to get some from GW150914, the first direct detection of an astrophysical gracitational-wave signal from a binary black hole system(BBH).

In [4]:
gps = gd.event_gps('GW150914')
print(gps)

1126259462.4


Now we can build a `[start, end)` GPS segment of 10 seconds around this time.

In [5]:
segment = (int(gps)-5, int(gps)+5)
print(segment)

(1126259457, 1126259467)


We can use the appropriate identifier to retrieve the data. For example, I'll choose LIGO-Livingston Interferometer to query for full data.
* `G1` : GEO600
* `H1` : LIGO Hanford
* `L1` : LIGO Livingston
* `V1` : (Advanced) VIRGO
* `K1` : KAGRA (Japanese observatory will come online in future)
 

In [6]:
ldata = gpt.TimeSeries.fetch_open_data('L1', *segment, verbose = True)
ldata

Fetched 1 URLs from www.gw-openscience.org for [1126259457 .. 1126259467))
Reading data... [Done]


<TimeSeries([-9.31087178e-19, -9.75697062e-19, -1.01074940e-18,
             ..., -1.13460681e-18, -1.11414494e-18,
             -1.15931435e-18]
            unit=Unit(dimensionless),
            t0=<Quantity 1.12625946e+09 s>,
            dt=<Quantity 0.00024414 s>,
            name='Strain',
            channel=None)>

In [7]:
try:
    vdata = gpt.TimeSeries.fetch_open_data('V1', *segment, verbose = True)
    print('found')
    vdata
except ValueError:
    print(f"Can't find the dataset for V1 covering [{segment[0]}, {segment[1]})")

Can't find the dataset for V1 covering [1126259457, 1126259467)


Now, let's plot the timeseries data

In [8]:
lData = go.Scatter(
    x = np.linspace(0, 10, len(ldata)),
    y = ldata,
    mode = 'lines',
    line = dict(
        shape = 'spline',
        color = 'midnightblue'
    )
)
layout = go.Layout(
    title = f'Time Series plot as detected by L1 for 10 seconds around {gps} GPS time',
    xaxis_title = f'Time(s) from {gtm.tconvert(segment[0])} to {gtm.tconvert(segment[1])} UTC',
    yaxis_title = 'Observation'
)
fig = go.Figure(data = [lData], layout = layout)
fig.show()

## Handling data in frequency domain using Fourier transform
Using Fourier transform, can see which frequencies contain lots of power and which have less.\
One can calculate the Fourier transform of `TimeSeries` using the method `fft()`.

In [9]:
ldataFFT = ldata.fft()
ldataFFT

<FrequencySeries([-1.05219333e-18+0.00000000e+00j,
                  -9.36314427e-22+6.61514979e-23j,
                  -8.74693847e-22-3.26717194e-23j, ...,
                   6.63325463e-24-1.10870240e-26j,
                   6.73663868e-24-7.82178218e-26j,
                   6.69598891e-24+0.00000000e+00j]
                 unit=Unit(dimensionless),
                 f0=<Quantity 0. Hz>,
                 df=<Quantity 0.1 Hz>,
                 epoch=<Time object: scale='utc' format='gps' value=1126259457.0>,
                 name='Strain',
                 channel=None)>

Now, the above Fourier transform is in complex form. One can use `abs()` to extract the amplitude and plot that.

In [10]:
lDataFFT = go.Scatter(
    y = ldataFFT.abs(),
    mode = 'lines',
    line = dict(
        shape = 'spline',
        color = 'limegreen'
    )
)
layout = go.Layout(
    title = 'Frequency Plot',
    xaxis = dict(
        title = 'Frequency [Hz]',
        type = 'log'
    ),
    yaxis = dict(
        title = 'counts',
        type = 'log'
    )
)
fig = go.Figure(data = [lDataFFT], layout = layout)
fig.show()

The problem with the FFT is that it works under the assumption that the given data is periodic, which means the edges of the data look like dicontinuities when transformed. I need to apply a window function to the time-domain data before transforming, which can be done using `scipy.signal` module:

In [11]:
window = sc.signal.get_window('hann', ldata.size)
lWindow = ldata * window 

In [12]:
ampFft = lWindow.fft().abs()

In [13]:
AmpFFT = go.Scatter(
    y = ampFft,
    mode = 'lines',
    line = dict(
        shape = 'spline',
        color = 'hotpink'
    )
)
layout = go.Layout(
    title = 'Rectified Frequency Plot',
    xaxis = dict(
        title = 'Frequency [Hz]',
        type = 'log'
    ),
    yaxis = dict(
        title = 'counts',
        type = 'log'
    )
)
fig = go.Figure(data = [AmpFFT], layout = layout)
fig.show()

This looks a little more like what is expectd for the amplitude spectral density of a gravitational-wave detector.

---
## Calculating the power spectral density
---
In practice, a large number of FFTs is used to estimate an average power spectral density over a long period of data. I'll use `asd()` method, which uses Welch's method to combine FFTs of overlapping, windowed chunks of data.

In [14]:
PSD = ldata.asd(fftlength = 5, method = 'median')
PowerSpectralDensity = go.Scatter(
    y = PSD,
    mode = 'lines',
    line = dict(
        shape = 'spline',
        color = 'darkcyan'
    )
)
layout = go.Layout(
    title = 'Power Spectral Density Plot',
    xaxis = dict(
        title = 'Frequency [Hz]',
        type = 'log'
    ),
    yaxis = dict(
        title = r'[$Hz^{-1/2}$]',
        type = 'log'
    )
)
fig = go.Figure(data = [PowerSpectralDensity], layout = layout)
fig.show()

In [15]:
fig.update_layout(xaxis = dict(range = [3+np.log10(2.1), 3.9]))
fig.show()

The ASD is a standard tool used to study the frquency-domain sensitivity of a gravitational-wave detector. For the LIGO-Livingston data I loaded, I can see large spikes at certain frequencies:


*   ~2500 Hz
*   ~5000 Hz
*   ~7500 Hz\
Loading more data allows more FFTs to be averaged during the ASD calculation, meaning the random variations get averaged out, and one can observe more detail:


In [16]:
tSegment = (int(gps)-512, int(gps)+512)
try:
    ldata2 = gpt.TimeSeries.fetch_open_data('L1', *tSegment, cache=True)
except ValueError:
    print("Can't find the data from L1 within the given segment")

In [17]:
lAsd2 = ldata2.asd(fftlength = 4, method = 'median')

In [18]:
lASD2 = go.Scatter(
    y = lAsd2,
    mode = 'lines',
    name = 'LIGO-Livingston',
    line = dict(
        shape = 'spline',
        color = 'mediumpurple'
    )
)
layout = go.Layout(
    title = f'LIGO Frequency plot from {tSegment[0]} to {tSegment[1]} GPS seconds',
    xaxis = dict(
        title = 'Frequency [Hz]',
        type = 'log',
        range = [2, 3+np.log10(7)]
    ),
    yaxis = dict(
        title = r'[$Hz^{-1/2}$]',
        type = 'log',
        range = [-24, -20]
    )
)
fig = go.Figure(data = [lASD2], layout = layout)
fig.show()

In [19]:
fig.update_layout(xaxis = dict(range = [2 + np.log10(2), 2 + np.log10(8)]))

So, some more features can be seen, including sets of lines around ~240 Hz, ~290 Hz and ~720 Hz.\
For comparison, I'll load the LIGO-Hanford data and plot that as well:

In [20]:
try:
    hdata2 = gpt.TimeSeries.fetch_open_data('H1', *tSegment, cache=True)
except ValueError:
    print("Can't find the data from H1 within the given segment")

hAsd2 = hdata2.asd(fftlength = 4, method = "median")

In [21]:
hASD2 = go.Scatter(
    y = hAsd2,
    name = 'LIGO-Hanford',
    mode = 'lines',
    line = dict(
        shape = 'spline',
        color = 'hotpink',
    )
)
layout = go.Layout(
    title = f'LIGO Frequency plot from {tSegment[0]} to {tSegment[1]} GPS seconds',
    xaxis = dict(
        title = 'Frequency [Hz]',
        type = 'log',
        range = [2, 3+np.log10(9)]
    ),
    yaxis = dict(
        title = r'[$Hz^{-1/2}$]',
        type = 'log',
        range = [-24, -20]
    )
)

In [22]:
fig = go.Figure(data = [lASD2, hASD2], layout = layout)
fig.show()
# fig.update_layout(xaxis = dict(range = [2, 3+np.log10(7)]))